<a href="https://colab.research.google.com/github/lemarunico2020/cyberanalyst_nextgen/blob/main/ingesta_IoC_MISP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Playbook para Ingesta de IOCs en MISP

Este playbook guía paso a paso a analistas de ciberseguridad para ingestar Indicadores de Compromiso (IOCs) en una instancia MISP de manera profesional y segura.

## Requisitos Previos

1. Tener acceso a una instancia MISP con una clave API
2. Un archivo de texto con IOCs (uno por línea)
3. Un archivo de texto con credenciales MISP
4. Python 3.6+ y las bibliotecas que instalaremos

## Estructura de los archivos de entrada

### Archivo de IOCs (`iocs.txt`)
Debe contener un IOC por línea. Opcionalmente, puede incluir un tipo y comentario separados por comas:

* 8.8.8.8
* malware-domain.com
* 8175486c21a5376dd9a5cd14edd6af070b4f6e50c10e9b4d881881532b42261e
* ip-src,192.168.1.1,IP interna sospechosa
* url,https://malicious-site.com/payload.php,Página de descarga de malware


### Archivo de credenciales (`misp_config.txt`)
Debe contener las siguientes líneas:

* url=https://your-misp-instance.com
* key=YourAPIKey
* verifycert=False




---



## Paso 1: Instalación de dependencias necesarias

Primero, instalemos las bibliotecas necesarias para trabajar con MISP y validar IOCs:

In [1]:
!pip install pymisp python-dateutil validators ipaddress tqdm colorama PyYAML

## Paso 2: Importar las bibliotecas requeridas

In [2]:
import os
import re
import sys
import json
import uuid
import hashlib
import ipaddress
import datetime
import csv
import yaml
from pathlib import Path
from urllib.parse import urlparse
import logging
from tqdm.notebook import tqdm
from colorama import Fore, Style, init
import validators
from collections import defaultdict, Counter

# Inicializar colorama para mostrar colores en la salida
init(autoreset=True)

# Importar PyMISP
try:
    from pymisp import PyMISP, MISPEvent, MISPAttribute, MISPObject, MISPTag
    print(f"{Fore.GREEN}✓ PyMISP importado correctamente{Style.RESET_ALL}")
except ImportError:
    print(f"{Fore.RED}✗ Error: No se pudo importar PyMISP. Asegúrese de haberlo instalado correctamente.{Style.RESET_ALL}")
    sys.exit(1)

# Configurar el sistema de registro (logging)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("misp_ioc_ingestion.log"),
        logging.StreamHandler()
    ]
)

✓ PyMISP importado correctamente


## Paso 3: Definir constantes y variables de utilidad

In [3]:
# Mapeo de extensiones de archivos a tipos MISP
IOC_TYPE_MAPPING = {
    'ip': ['ip-src', 'ip-dst'],
    'domain': ['domain', 'hostname'],
    'url': ['url'],
    'email': ['email-src', 'email-dst'],
    'md5': ['md5'],
    'sha1': ['sha1'],
    'sha256': ['sha256'],
    'sha512': ['sha512'],
    'filename': ['filename'],
    'filepath': ['filename|sha256', 'filename|md5'],
    'registry': ['regkey', 'regkey|value'],
    'mutex': ['mutex'],
    'cve': ['vulnerability'],
    'malware': ['text'],
    'cidr': ['ip-src', 'ip-dst'],
    'ja3': ['ja3-fingerprint-md5'],
    'ja3s': ['ja3-fingerprint-md5'],
    'ssdeep': ['ssdeep'],
    'imphash': ['imphash'],
    'threat-actor': ['threat-actor'] # Añadido soporte para threat actors
}

# Patrones de expresiones regulares para validar diferentes tipos de IOCs
IOC_PATTERNS = {
    'md5': re.compile(r'^[a-fA-F0-9]{32}$'),
    'sha1': re.compile(r'^[a-fA-F0-9]{40}$'),
    'sha256': re.compile(r'^[a-fA-F0-9]{64}$'),
    'sha512': re.compile(r'^[a-fA-F0-9]{128}$'),
    'ip': re.compile(r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$'),
    'email': re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'),
    'url': re.compile(r'^(https?|ftp)://[^\s/$.?#].[^\s]*$'),
    'domain': re.compile(r'^(?:[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?\.)+[a-zA-Z]{2,}$'),
    'cidr': re.compile(r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}/[0-9]{1,2}$'),
    'cve': re.compile(r'^CVE-\d{4}-\d{4,}$', re.IGNORECASE),
    'ja3': re.compile(r'^[a-fA-F0-9]{32}$'),  # Same as MD5
    'ja3s': re.compile(r'^[a-fA-F0-9]{32}$'),  # Same as MD5
    'ssdeep': re.compile(r'^\d+:[a-zA-Z0-9/+]+:[a-zA-Z0-9/+]+$'),
    'imphash': re.compile(r'^[a-fA-F0-9]{32}$'),  # Same as MD5
    # No hay un patrón específico para threat actors, se detectarán por contexto
}

# Tipos MISP predeterminados para tipos detectados automáticamente
DEFAULT_MISP_TYPES = {
    'md5': 'md5',
    'sha1': 'sha1',
    'sha256': 'sha256',
    'sha512': 'sha512',
    'ip': 'ip-src',
    'email': 'email-src',
    'url': 'url',
    'domain': 'domain',
    'cidr': 'ip-src',
    'cve': 'vulnerability',
    'ja3': 'ja3-fingerprint-md5',
    'ja3s': 'ja3-fingerprint-md5',
    'ssdeep': 'ssdeep',
    'imphash': 'imphash',
    'threat-actor': 'threat-actor'  # Añadido soporte para threat actors
}

# Estado para el seguimiento de estadísticas
stats = {
    'total_iocs': 0,
    'valid_iocs': 0,
    'invalid_iocs': 0,
    'added_to_misp': 0,
    'failed_to_add': 0,
    'by_type': Counter(),
    'invalid_by_type': Counter(),
    'start_time': datetime.datetime.now()
}

## Paso 4: Definir funciones de utilidad

A continuación, definiremos funciones para cargar la configuración MISP, detectar y validar tipos de IOC, y procesar los IOCs.

In [4]:
def load_misp_config(config_file):
    """
    Carga la configuración MISP desde un archivo de texto.

    Args:
        config_file (str): Ruta al archivo de configuración

    Returns:
        dict: Configuración MISP con url, key y verifycert
    """
    config = {}
    required_keys = ['url', 'key']

    try:
        with open(config_file, 'r') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue

                if '=' in line:
                    key, value = line.split('=', 1)
                    config[key.strip()] = value.strip()

        # Verificar claves requeridas
        for key in required_keys:
            if key not in config:
                raise ValueError(f"Clave requerida '{key}' no encontrada en el archivo de configuración.")

        # Valores predeterminados
        if 'verifycert' not in config:
            config['verifycert'] = 'False'

        # Convertir verifycert a booleano
        config['verifycert'] = config['verifycert'].lower() in ('true', 'yes', '1')

        return config
    except Exception as e:
        logging.error(f"Error al cargar la configuración MISP: {e}")
        raise

def detect_ioc_type(ioc):
    """
    Detecta automáticamente el tipo de IOC basado en su formato.

    Args:
        ioc (str): El indicador a analizar

    Returns:
        str: Tipo de IOC detectado o None si no se puede determinar
    """
    ioc = ioc.strip()

    # Verificar si es un CVE
    if re.match(r'^CVE-\d{4}-\d{4,}$', ioc, re.IGNORECASE):
        return 'cve'

    # Comprobar cada patrón
    for ioc_type, pattern in IOC_PATTERNS.items():
        if pattern.match(ioc):
            # Verificaciones adicionales para algunos tipos
            if ioc_type == 'ip':
                try:
                    ip = ipaddress.ip_address(ioc)
                    return 'ip'
                except ValueError:
                    continue
            elif ioc_type == 'cidr':
                try:
                    network = ipaddress.ip_network(ioc)
                    return 'cidr'
                except ValueError:
                    continue
            elif ioc_type == 'url':
                if validators.url(ioc):
                    return 'url'
                continue
            elif ioc_type == 'domain':
                if validators.domain(ioc):
                    return 'domain'
                continue
            elif ioc_type == 'email':
                if validators.email(ioc):
                    return 'email'
                continue

            return ioc_type

    # Si no se detectó ningún tipo conocido
    return None

def validate_ioc(ioc_type, ioc_value):
    """
    Valida si un IOC tiene la estructura correcta según su tipo.

    Args:
        ioc_type (str): Tipo de IOC (ip, md5, url, etc.)
        ioc_value (str): Valor del IOC a validar

    Returns:
        bool: True si el IOC es válido, False en caso contrario
    """
    if not ioc_type or not ioc_value:
        return False

    ioc_value = ioc_value.strip()

    # Validaciones especiales por tipo
    if ioc_type in ['ip-src', 'ip-dst']:
        try:
            ipaddress.ip_address(ioc_value)
            return True
        except ValueError:
            return False
    elif ioc_type == 'domain':
        return validators.domain(ioc_value)
    elif ioc_type == 'url':
        return validators.url(ioc_value)
    elif ioc_type == 'email-src' or ioc_type == 'email-dst' or ioc_type == 'email':
        return validators.email(ioc_value)
    elif ioc_type == 'md5':
        return bool(re.match(r'^[a-fA-F0-9]{32}$', ioc_value))
    elif ioc_type == 'sha1':
        return bool(re.match(r'^[a-fA-F0-9]{40}$', ioc_value))
    elif ioc_type == 'sha256':
        return bool(re.match(r'^[a-fA-F0-9]{64}$', ioc_value))
    elif ioc_type == 'sha512':
        return bool(re.match(r'^[a-fA-F0-9]{128}$', ioc_value))
    elif ioc_type == 'vulnerability':
        return bool(re.match(r'^CVE-\d{4}-\d{4,}$', ioc_value, re.IGNORECASE))
    elif ioc_type == 'ja3-fingerprint-md5':
        return bool(re.match(r'^[a-fA-F0-9]{32}$', ioc_value))
    elif ioc_type == 'ssdeep':
        return bool(re.match(r'^\d+:[a-zA-Z0-9/+]+:[a-zA-Z0-9/+]+$', ioc_value))
    elif ioc_type == 'imphash':
        return bool(re.match(r'^[a-fA-F0-9]{32}$', ioc_value))
    elif ioc_type == 'threat-actor':
        # Cualquier texto no vacío es válido como threat actor
        return bool(ioc_value.strip())

    # Para otros tipos, asumir válido
    return True

def parse_ioc_line(line):
    """
    Parsea una línea del archivo de IOCs.
    Formatos soportados:
    - valor
    - tipo,valor
    - tipo,valor,comentario
    - Etiqueta: valor
    - CVE-XXXX-XXXX (formato CVE)
    - Patrones de threat actor

    Args:
        line (str): Línea a procesar

    Returns:
        tuple: (tipo, valor, comentario) o (None, None, None) si inválido
    """
    line = line.strip()
    if not line or line.startswith('#'):
        return None, None, None

    # Ignorar líneas que parecen ser encabezados
    if not any(char in line for char in ",:/.@0123456789-"):
        return None, None, None

    # Verificar si es un CVE directamente
    if re.match(r'^CVE-\d{4}-\d{4,}$', line, re.IGNORECASE):
        return 'vulnerability', line.upper(), ""

    # Buscar patrones como "CVE: CVE-2023-12345" o "Vulnerabilidad: CVE-2023-12345"
    if ': ' in line and ('cve' in line.lower() or 'vulnerabilidad' in line.lower() or 'vulnerability' in line.lower()):
        parts = line.split(': ', 1)
        if len(parts) == 2 and re.match(r'^CVE-\d{4}-\d{4,}$', parts[1].strip(), re.IGNORECASE):
            return 'vulnerability', parts[1].strip().upper(), ""

    # Patrones para threat actors
    if ': ' in line and ('threat' in line.lower() or 'actor' in line.lower() or 'grupo' in line.lower() or 'atacante' in line.lower()):
        parts = line.split(': ', 1)
        if len(parts) == 2 and parts[1].strip():
            return 'threat-actor', parts[1].strip(), ""

    # Comprobar formato "Etiqueta: valor" para otros tipos
    if ': ' in line and ',' not in line:
        parts = line.split(': ', 1)
        if len(parts) == 2:
            label, value = parts
            # Mapear etiquetas comunes a tipos MISP
            label_lower = label.lower()
            if 'md5' in label_lower:
                return 'md5', value.strip(), ""
            elif 'sha1' in label_lower:
                return 'sha1', value.strip(), ""
            elif 'sha256' in label_lower:
                return 'sha256', value.strip(), ""
            elif 'nombre del archivo' in label_lower or 'filename' in label_lower:
                return 'filename', value.strip(), ""
            elif 'ip' in label_lower:
                return 'ip-src', value.strip(), ""
            elif 'dominio' in label_lower or 'domain' in label_lower:
                return 'domain', value.strip(), ""
            elif 'url' in label_lower:
                return 'url', value.strip(), ""
            elif 'email' in label_lower or 'correo' in label_lower:
                return 'email-src', value.strip(), ""

    # Comprobar si es solo una URL (comienza con http o https)
    if line.startswith(('http://', 'https://')):
        return 'url', line, ""

    # Dividir por comas para diferentes formatos estándar
    parts = [p.strip() for p in line.split(',', 2)]

    if len(parts) == 1:  # Solo valor, detectar tipo
        value = parts[0]
        detected_type = detect_ioc_type(value)
        if detected_type:
            ioc_type = DEFAULT_MISP_TYPES.get(detected_type)
            return ioc_type, value, ""
        else:
            return None, value, ""  # Tipo desconocido

    elif len(parts) == 2:  # tipo,valor
        ioc_type, value = parts
        # Manejo especial para threat-actor
        if ioc_type.lower() == 'threat-actor' or ioc_type.lower() == 'actor':
            return 'threat-actor', value, ""
        return ioc_type, value, ""

    elif len(parts) >= 3:  # tipo,valor,comentario
        ioc_type, value, comment = parts[0], parts[1], parts[2]
        # Manejo especial para threat-actor
        if ioc_type.lower() == 'threat-actor' or ioc_type.lower() == 'actor':
            return 'threat-actor', value, comment
        return ioc_type, value, comment

    return None, None, None

def add_context_to_ioc(ioc_type, ioc_value):
    """
    Añade información de contexto a un IOC basado en su tipo y valor.

    Args:
        ioc_type (str): Tipo de IOC
        ioc_value (str): Valor del IOC

    Returns:
        dict: Información de contexto
    """
    context = {}

    # Añadir contexto específico por tipo
    if ioc_type in ['ip-src', 'ip-dst']:
        try:
            ip = ipaddress.ip_address(ioc_value)
            context['is_private'] = ip.is_private
            context['is_global'] = ip.is_global
            context['is_loopback'] = ip.is_loopback
            context['version'] = f'IPv{ip.version}'
        except Exception:
            pass

    elif ioc_type == 'url':
        try:
            parsed = urlparse(ioc_value)
            context['scheme'] = parsed.scheme
            context['netloc'] = parsed.netloc
            context['path'] = parsed.path
            context['has_query'] = bool(parsed.query)
        except Exception:
            pass

    elif ioc_type == 'domain':
        # Contar niveles de dominio
        parts = ioc_value.split('.')
        context['tld'] = parts[-1] if len(parts) > 1 else ''
        context['domain_levels'] = len(parts)

    elif ioc_type == 'vulnerability':
        # Extraer año del CVE
        match = re.match(r'^CVE-(\d{4})-\d+$', ioc_value, re.IGNORECASE)
        if match:
            context['year'] = match.group(1)

    elif ioc_type == 'threat-actor':
        # No hay mucho contexto automático que añadir para threat actors
        context['type'] = 'Threat Actor'

    return context

## Paso 5: Cargar configuración MISP

A continuación, cargaremos la configuración MISP desde el archivo local:

In [5]:
# Archivo de configuración MISP (modificar según sea necesario)
misp_config_file = '/content/misp_config.txt'

try:
    # Cargar configuración
    misp_config = load_misp_config(misp_config_file)
    print(f"{Fore.GREEN}✓ Configuración MISP cargada correctamente{Style.RESET_ALL}")
    print(f"  URL: {misp_config['url']}")
    print(f"  Verificar certificado: {misp_config['verifycert']}")
except Exception as e:
    print(f"{Fore.RED}✗ Error al cargar la configuración MISP: {e}{Style.RESET_ALL}")
    print(f"  Asegúrese de que el archivo '{misp_config_file}' existe y tiene el formato correcto.")
    sys.exit(1)

✓ Configuración MISP cargada correctamente
  URL: https://misppriv.circl.lu
  Verificar certificado: False


## Paso 6: Inicializar conexión con MISP

Ahora, inicializaremos la conexión con la instancia MISP utilizando las credenciales cargadas:

In [ ]:
# Suprimir advertencias de SSL si verifycert está desactivado
if not misp_config['verifycert']:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

try:
    # Inicializar PyMISP
    misp = PyMISP(misp_config['url'], misp_config['key'], misp_config['verifycert'])

    # Verificar la conexión mediante una consulta simple
    # Buscar un evento (limitado a 1) solo para probar la conexión
    test_result = misp.search_index(limit=1)

    # Si llegamos aquí sin errores, la conexión funciona
    print(f"{Fore.GREEN}✓ Conexión exitosa con MISP{Style.RESET_ALL}")
    print(f"  API accesible en {misp_config['url']}")
    connection_success = True

except Exception as e:
    print(f"{Fore.RED}✗ Error al conectar con MISP: {e}{Style.RESET_ALL}")
    print("  Verifique las credenciales y la URL de la instancia MISP.")
    connection_success = False

# En lugar de sys.exit(), usamos esta bandera para control de flujo
if not connection_success:
    print("No se pudo establecer conexión con MISP. Resuelva el problema antes de continuar.")

## Paso 7: Leer y validar IOCs desde archivo local

A continuación, leeremos los IOCs desde el archivo local y validaremos su estructura:

In [ ]:
# Archivo de IOCs (modificar según sea necesario)
iocs_file = '/content/iocs.txt'

valid_iocs = []
invalid_iocs = []

try:
    with open(iocs_file, 'r') as f:
        lines = f.readlines()

    stats['total_iocs'] = len([l for l in lines if l.strip() and not l.strip().startswith('#')])
    print(f"\nLeyendo {stats['total_iocs']} IOCs desde {iocs_file}...")

    for line in tqdm(lines, desc="Validando IOCs"):
        line = line.strip()
        if not line or line.startswith('#'):
            continue

        # Parsear línea
        ioc_type, ioc_value, comment = parse_ioc_line(line)

        if not ioc_value:
            continue

        # Si el tipo no se detectó o especificó
        if not ioc_type:
            logging.warning(f"No se pudo determinar el tipo para: {ioc_value}")
            invalid_iocs.append({
                'value': ioc_value,
                'reason': 'Tipo desconocido',
                'line': line
            })
            stats['invalid_iocs'] += 1
            stats['invalid_by_type']['unknown'] += 1
            continue

        # Validar estructura del IOC
        if validate_ioc(ioc_type, ioc_value):
            # Añadir contexto
            context = add_context_to_ioc(ioc_type, ioc_value)

            valid_iocs.append({
                'type': ioc_type,
                'value': ioc_value,
                'comment': comment,
                'context': context
            })
            stats['valid_iocs'] += 1
            stats['by_type'][ioc_type] += 1
        else:
            logging.warning(f"IOC inválido: {ioc_value} (tipo: {ioc_type})")
            invalid_iocs.append({
                'type': ioc_type,
                'value': ioc_value,
                'reason': 'Estructura inválida',
                'line': line
            })
            stats['invalid_iocs'] += 1
            stats['invalid_by_type'][ioc_type] += 1

    # Mostrar resumen de validación
    print(f"\n{Fore.GREEN}✓ Validación completada{Style.RESET_ALL}")
    print(f"  IOCs totales: {stats['total_iocs']}")
    print(f"  IOCs válidos: {stats['valid_iocs']}")
    print(f"  IOCs inválidos: {stats['invalid_iocs']}")
    print("\nDistribución por tipo:")
    for ioc_type, count in stats['by_type'].most_common():
        print(f"  {ioc_type}: {count}")

    if invalid_iocs:
        print(f"\n{Fore.YELLOW}! IOCs inválidos:{Style.RESET_ALL}")
        for i, invalid in enumerate(invalid_iocs[:5], 1):
            print(f"  {i}. {invalid['value']} - {invalid['reason']}")

        if len(invalid_iocs) > 5:
            print(f"  ... y {len(invalid_iocs) - 5} más")

except FileNotFoundError:
    print(f"{Fore.RED}✗ Error: No se encontró el archivo '{iocs_file}'{Style.RESET_ALL}")
    sys.exit(1)
except Exception as e:
    print(f"{Fore.RED}✗ Error al procesar los IOCs: {e}{Style.RESET_ALL}")
    logging.exception("Error al procesar los IOCs")
    sys.exit(1)

## Paso 8: Crear evento en MISP

Ahora crearemos un evento en MISP para contener los IOCs validados:

In [ ]:
# Configurar información del evento
event_info = input("Ingrese un título descriptivo para el evento (o presione Enter para usar el predeterminado): ").strip()
if not event_info:
    event_info = f"IOCs importados automáticamente - {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}"

# Seleccionar distribución
print("\nNiveles de distribución:")
print("0: Tu organización solamente (predeterminado)")
print("1: Esta comunidad solamente")
print("2: Comunidades conectadas")
print("3: Todas las comunidades")
distribution = input("Seleccione nivel de distribución (0-3): ").strip()
distribution = int(distribution) if distribution.isdigit() and 0 <= int(distribution) <= 3 else 0

# Seleccionar nivel de amenaza
print("\nNiveles de amenaza:")
print("1: Alto")
print("2: Medio")
print("3: Bajo")
print("4: Indefinido (predeterminado)")
threat_level_id = input("Seleccione nivel de amenaza (1-4): ").strip()
threat_level_id = int(threat_level_id) if threat_level_id.isdigit() and 1 <= int(threat_level_id) <= 4 else 4

# Seleccionar nivel de análisis
print("\nNiveles de análisis:")
print("0: Inicial")
print("1: En curso")
print("2: Completado (predeterminado)")
analysis = input("Seleccione nivel de análisis (0-2): ").strip()
analysis = int(analysis) if analysis.isdigit() and 0 <= int(analysis) <= 2 else 2

try:
    # Crear evento
    event = MISPEvent()
    event.info = event_info
    event.distribution = distribution
    event.threat_level_id = threat_level_id
    event.analysis = analysis

    print(f"\nCreando evento en MISP: '{event_info}'...")
    response = misp.add_event(event)

    if isinstance(response, dict) and 'Event' in response:
        event_id = response['Event']['id']
        print(f"{Fore.GREEN}✓ Evento creado correctamente con ID: {event_id}{Style.RESET_ALL}")
    else:
        print(f"{Fore.RED}✗ Error al crear evento: Respuesta inesperada{Style.RESET_ALL}")
        print(f"  Respuesta: {response}")
        sys.exit(1)
except Exception as e:
    print(f"{Fore.RED}✗ Error al crear evento: {e}{Style.RESET_ALL}")
    logging.exception("Error al crear evento en MISP")
    sys.exit(1)

## Paso 9: Añadir etiquetas al evento (NO POR EL MOMENTO)

Añadiremos etiquetas útiles al evento para facilitar su clasificación:

In [ ]:
# Conjunto de etiquetas recomendadas
suggested_tags = [
    {'name': 'tlp:amber', 'description': 'TLP:AMBER - Difusión limitada'},
    {'name': 'type:OSINT', 'description': 'Información de fuentes abiertas'},
    {'name': 'misp:import="automatic"', 'description': 'Importado automáticamente'},
    {'name': 'source:local-file', 'description': 'Fuente: archivo local'}
]

# Mostrar etiquetas recomendadas
print("\nEtiquetas recomendadas:")
for i, tag in enumerate(suggested_tags, 1):
    print(f"  {i}. {tag['name']} - {tag['description']}")

print("\nSeleccione etiquetas a aplicar (separadas por comas, por ejemplo: 1,2,4)")
print("Presione Enter para aplicar todas las etiquetas recomendadas")
selected = input("Selección: ").strip()

# Procesar selección
if not selected:
    tags_to_apply = [tag['name'] for tag in suggested_tags]
else:
    try:
        indices = [int(idx.strip()) - 1 for idx in selected.split(',')]
        tags_to_apply = [suggested_tags[idx]['name'] for idx in indices if 0 <= idx < len(suggested_tags)]
    except Exception:
        print(f"{Fore.YELLOW}! Selección inválida, aplicando todas las etiquetas recomendadas{Style.RESET_ALL}")
        tags_to_apply = [tag['name'] for tag in suggested_tags]

# Añadir etiqueta personalizada
custom_tag = input("\nIngrese etiqueta personalizada (opcional): ").strip()
if custom_tag:
    tags_to_apply.append(custom_tag)

# Aplicar etiquetas
print(f"\nAplicando {len(tags_to_apply)} etiquetas al evento...")
for tag_name in tqdm(tags_to_apply, desc="Aplicando etiquetas"):
    try:
        misp.tag(event_id, tag_name)
    except Exception as e:
        print(f"{Fore.YELLOW}! Error al aplicar etiqueta '{tag_name}': {e}{Style.RESET_ALL}")

print(f"{Fore.GREEN}✓ Etiquetas aplicadas correctamente{Style.RESET_ALL}")

## Paso 10: Añadir IOCs al evento

Ahora añadiremos los IOCs validados al evento:

In [ ]:
print(f"\nAñadiendo {len(valid_iocs)} IOCs validados al evento...")

# Configurar to_ids predeterminado
to_ids_default = True
to_ids_input = input("¿Marcar IOCs para sistemas de detección? (S/n): ").strip().lower()
if to_ids_input and to_ids_input[0] == 'n':
    to_ids_default = False

failed_iocs = []

for ioc in tqdm(valid_iocs, desc="Añadiendo IOCs"):
    try:
        # Preparar comentario
        comment = ioc['comment']

        # Añadir información de contexto al comentario si está disponible
        if ioc['context']:
            context_str = "; ".join(f"{k}={v}" for k, v in ioc['context'].items())
            if comment:
                comment += f" | {context_str}"
            else:
                comment = context_str

        # Crear atributo
        attribute = {
            'type': ioc['type'],
            'value': ioc['value'],
            'comment': comment,
            'to_ids': to_ids_default
        }

        # Añadir atributo al evento
        result = misp.add_attribute(event_id, attribute)

        if isinstance(result, dict) and 'Attribute' in result:
            stats['added_to_misp'] += 1
        else:
            failed_iocs.append({
                'ioc': ioc,
                'reason': f"Respuesta inesperada: {result}"
            })
            stats['failed_to_add'] += 1

    except Exception as e:
        failed_iocs.append({
            'ioc': ioc,
            'reason': str(e)
        })
        stats['failed_to_add'] += 1
        logging.warning(f"Error al añadir IOC {ioc['value']}: {e}")

# Publicar el evento
try:
    misp.publish(event_id)
    print(f"{Fore.GREEN}✓ Evento publicado correctamente{Style.RESET_ALL}")
except Exception as e:
    print(f"{Fore.YELLOW}! Error al publicar evento: {e}{Style.RESET_ALL}")
    print("  El evento se ha creado pero puede estar en estado borrador")

# Mostrar resumen
print(f"\n{Fore.GREEN}✓ Proceso completado{Style.RESET_ALL}")
print(f"  IOCs agregados correctamente: {stats['added_to_misp']}")
print(f"  IOCs fallidos: {stats['failed_to_add']}")
print(f"  Tiempo total de procesamiento: {datetime.datetime.now() - stats['start_time']}")

# Mostrar enlace al evento
event_url = f"{misp_config['url']}/events/view/{event_id}"
print(f"\nVer evento en MISP: {event_url}")

## Paso 11: Guardar informe detallado

Finalmente, guardaremos un informe detallado de la operación:

In [ ]:
# Generar informe
report_file = f"misp_ingestion_report_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.yaml"

report = {
    'timestamp': datetime.datetime.now().isoformat(),
    'event': {
        'id': event_id,
        'info': event_info,
        'url': event_url
    },
    'stats': {
        'total_iocs': stats['total_iocs'],
        'valid_iocs': stats['valid_iocs'],
        'invalid_iocs': stats['invalid_iocs'],
        'added_to_misp': stats['added_to_misp'],
        'failed_to_add': stats['failed_to_add'],
        'processing_time': str(datetime.datetime.now() - stats['start_time']),
        'by_type': dict(stats['by_type'])
    },
    'tags_applied': tags_to_apply,
    'invalid_iocs': [
        {'value': ioc['value'], 'reason': ioc['reason']} for ioc in invalid_iocs
    ],
    'failed_iocs': [
        {'value': f['ioc']['value'], 'type': f['ioc']['type'], 'reason': f['reason']} for f in failed_iocs
    ]
}

try:
    with open(report_file, 'w') as f:
        yaml.dump(report, f, default_flow_style=False)
    print(f"{Fore.GREEN}✓ Informe guardado en {report_file}{Style.RESET_ALL}")
except Exception as e:
    print(f"{Fore.YELLOW}! Error al guardar informe: {e}{Style.RESET_ALL}")

## Resumen y Recomendaciones

¡Felicidades! Ha completado exitosamente la ingesta de IOCs en MISP. Aquí hay algunas recomendaciones para el seguimiento:

1. **Verificación**: Visite la URL del evento proporcionada para verificar que todos los IOCs se han importado correctamente.

2. **Revisión de IOCs fallidos**: Si hubo IOCs fallidos, revise el informe generado para entender las razones y corregirlos si es necesario.

3. **Mejora continua**: Considere guardar este notebook y personalizarlo según sus necesidades específicas. Puede añadir validaciones adicionales, integraciones con otras fuentes o automatizar el proceso mediante programación.

4. **Automatización**: Para ingestas regulares, considere programar la ejecución de este notebook utilizando herramientas como papermill o nbconvert junto con cron o el Programador de tareas de Windows.

5. **Enriquecimiento**: Considere añadir capacidades de enriquecimiento de IOCs utilizando servicios externos o información interna adicional.

6. **Documentación**: Mantenga documentación clara sobre el origen de los IOCs y el proceso de ingesta para referencia futura.

### Recursos Adicionales

- [Documentación oficial de MISP](https://www.misp-project.org/documentation/)
- [Repositorio de PyMISP](https://github.com/MISP/PyMISP)
- [Taxonomías MISP](https://github.com/MISP/misp-taxonomies) para etiquetado más efectivo
- [Galería de objetos MISP](https://github.com/MISP/misp-objects) para ingestas más estructuradas

### Notas Finales

Este playbook está diseñado para ser modular y adaptable. Puede extenderlo para cubrir casos de uso más específicos, como la ingesta desde otras fuentes o el enriquecimiento automático de IOCs. La estructura de validación y el informe detallado ayudan a mantener la calidad de los datos en su instancia MISP.